In [1]:
venue_ids = [
    'rec4maUwKspPcDxNl',
    'recFYiOZvNnYDgbfk',
    'recInFDrcQ4KutUcL',
    'recIrQ1v64zKajXU1',
    'recalzwrAJkBYOz8w',
]

In [2]:
import petl as etl
from utils.themes.programme import Programme

In [ ]:
def explode_event_reports(venue):
    for report in venue.event_reports:
        yield (venue.id, venue.name, report)

events = (
    etl.fromcolumns([venue_ids], header=['id'])
    .join(
        Programme.venues
        .cut('id', 'organisation_or_venue_name', 'event_reports')
        .rename({
            'organisation_or_venue_name': 'name',
        })
    )
    .rowmapmany(explode_event_reports, header=['venue_id', 'venue_name', 'report_id'])
    .selectnotin('report_id', ['rec7SCoXytLk1tbSj'])
    .leftjoin(
        (
            Programme.event_reports
            .cut(
                'id',
                'project_id',
                'Project Name',
                'Name',
                'event_date',
                'audience', 
            )
            .rename({
                'Name': 'event_name',
                'Project Name': 'project_name',
            })
        ), lkey='report_id', rkey='id'
    )
    .sort('event_date')
)

In [4]:
events

venue_id,venue_name,report_id,project_id,project_name,event_name,event_date,audience
recInFDrcQ4KutUcL,Loading Bay Gallery,rec1xFvwVx8kbfM7H,recB0Dk8oQgJWqBrz,BBC Extraordinary Portraits,BBC Extraordinary Portraits - 13/03/25,2025-03-13,356
recInFDrcQ4KutUcL,Loading Bay Gallery,rec5QHHkoh9gGqiIj,recB0Dk8oQgJWqBrz,BBC Extraordinary Portraits,BBC Extraordinary Portraits - 14/03/25,2025-03-14,458
recFYiOZvNnYDgbfk,Loading Bay Theatre,recoJvqjAB8QDI49z,rec0hlj028BZL1wIj,Loaded Laughs (#1),Loaded Laughs (#1) - 14/03/25,2025-03-14,164
recFYiOZvNnYDgbfk,Loading Bay Theatre,rec44Pc4XbpwVNw6M,recBCpcIwvaljBt32,Lady Maisery / O'Hooley & Tidow,Lady Maisery / O'Hooley & Tidow - 16/03/25,2025-03-16,183
recInFDrcQ4KutUcL,Loading Bay Gallery,recKDnBk0hVScpljE,recB0Dk8oQgJWqBrz,BBC Extraordinary Portraits,BBC Extraordinary Portraits - 16/03/25,2025-03-16,583


In [13]:
aggregated = (
    events
    .addfield('month', lambda r: r.event_date.replace(day=1))
    .aggregate(
        ['project_name', 'venue_name', 'month'],
        sum,
        'audience'
    )
)

# months = [m for m in set(aggregated.values('month'))]

(
    aggregated
    .recast(['project_name', 'venue_name'], variablefield='month')
    .replaceall(None, 0)
    # .addfield('TOTAL', lambda r: [f for f in months])
).displayall()

# months

project_name,venue_name,2025-03-01,2025-04-01,2025-05-01
BBC Extraordinary Portraits,Loading Bay Gallery,6538,3326,0
Big Brass Blowout,Loading Bay Theatre,0,242,0
Dracula - Black Eyed Theatre,Loading Bay Theatre,535,0,0
Frontline 1984-1985 by Victor Wedderburn,Loading Bay Gallery,0,1008,762
Lady Maisery / O'Hooley & Tidow,Loading Bay Theatre,183,0,0
Little Wimmin,Loading Bay Theatre,0,129,0
Loaded Laughs (#1),Loading Bay Theatre,164,0,0
The 56,Loading Bay Theatre,0,0,164
Toxic - Dibby Theatre,Loading Bay Theatre,0,165,0
Unspun - 509 Arts,Loading Bay Basement,1852,0,0


In [18]:
aggregated.aggregate(['project_name', 'venue_name'], sum, 'value').displayall()

project_name,venue_name,value
BBC Extraordinary Portraits,Loading Bay Gallery,9864
Big Brass Blowout,Loading Bay Theatre,242
Dracula - Black Eyed Theatre,Loading Bay Theatre,535
Frontline 1984-1985 by Victor Wedderburn,Loading Bay Gallery,1770
Lady Maisery / O'Hooley & Tidow,Loading Bay Theatre,183
Little Wimmin,Loading Bay Theatre,129
Loaded Laughs (#1),Loading Bay Theatre,164
The 56,Loading Bay Theatre,164
Toxic - Dibby Theatre,Loading Bay Theatre,165
Unspun - 509 Arts,Loading Bay Basement,1852


In [23]:
aggregated.aggregate(['venue_name', 'month'], sum, 'value').recast(variablefield='month').replaceall(None, 0).displayall()

venue_name,2025-03-01,2025-04-01,2025-05-01
Loading Bay Basement,1852,0,0
Loading Bay Gallery,6538,4334,762
Loading Bay Theatre,882,618,164
